In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/StockForcasting/Stock-Forcast')
from functions import *

Mounted at /content/drive


In [2]:
!ls

AdaBoostwith10features.ipynb  functions.py  __pycache__  usingCloseValues  Using.ipynb	usingOLHV


In [3]:
import pandas as pd
import numpy as np
import pickle
from functions import *
os.chdir('/content/drive/MyDrive/dataForStockForcast')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [4]:
time_steps = 32
x_train, x_valid, y_train, y_valid = preprocess_data_equal_division('training_set.pkl', le=True, num_stocks=60, time_steps=time_steps, only_close=False, equal_split=False)

In [5]:
x_train.shape, x_valid.shape

((104160, 32, 4), (26040, 32, 4))

In [ ]:
x_train = x_train.reshape((x_train.shape[0], -1))
x_valid = x_valid.reshape((x_valid.shape[0], -1))

In [ ]:
import xgboost as xg
xg_train = xg.DMatrix(x_train, y_train)
xg_valid = xg.DMatrix(x_valid)

In [ ]:
params = {'objective':'multi:softmax', 'eval_metric':'merror', 'num_class':'3',
          # 'device':'cuda',
          'eta':'0.1', 'max_depth':'5'}

In [ ]:
from tqdm import tqdm
models_list = []
for i in tqdm(range(1)):
  mdl = xg.train(params, xg_train, num_boost_round=70)
  models_list.append(mdl)

100%|██████████| 1/1 [00:11<00:00, 11.92s/it]


In [ ]:
def models_predict(model_list, x_values):
  preds = np.zeros((x_values.get_data().shape[0], ))
  for mdl in  models_list:
    preds += mdl.predict(x_values)
  return preds/len(model_list)

In [ ]:
train_preds = models_predict(models_list, xg_train)

In [ ]:
_, _, _, _, _ = metric_calculations(train_preds, y_train, set_ = "Train Set")

For Train Set
  Accuracy: 62.467357910906294,
  Precision: 62.66189568667107,
  Recall: 61.827461815949036,
  Specificity: 63.108139266774266,
  F1score: 62.24188220749869


In [ ]:
valid_preds = models_predict(models_list, xg_valid)

In [ ]:
_, _, _, _, _ = metric_calculations(valid_preds, y_valid, set_ = "Validation Set")

For Validation Set
  Accuracy: 50.33026113671275,
  Precision: 50.36758955107149,
  Recall: 49.43199263125575,
  Specificity: 51.229634183830306,
  F1score: 49.8954055938638


In [ ]:
pickle_file_path = 'testing_set1.pkl'
x_test, y_test = preprocess_data_equal_division(pickle_file_path, le=True, num_stocks=5, time_steps=32, only_close=False, split=False)

In [ ]:
x_test = x_test.reshape((x_test.shape[0], -1))

In [ ]:
xg_test = xg.DMatrix(x_test)

In [ ]:
test_preds = models_predict(models_list, xg_test)

In [ ]:
_, _, _, _, _ = metric_calculations(test_preds, y_test, set_ = "Test Set")

For Test Set
  Accuracy: 32.765460910151695,
  Precision: 31.08108108108108,
  Recall: 23.64774251229325,
  Specificity: 42.724609375,
  F1score: 26.859609037826857
